In [1]:
import pandas as pd
import geopandas as gpd

from bokeh.plotting import figure, save, show
from bokeh.models import ColumnDataSource
#For the Map Tiles
import xyzservices.providers as xyz
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
tile_provider = get_provider(xyz.OpenStreetMap.Mapnik)

import panel as pn
pn.extension()

from bokeh.io import output_notebook

import warnings

warnings.filterwarnings("ignore")

# READ THE DATA

In [2]:
df = pd.read_csv('data/Coffee Brands Footprint.csv',
                index_col=0)
df.head()

,id,brand,name,lat,lng,vicinity
1,ChIJ8aEd9AfyljMRtVgVdqt50kQ,Coffee Bean and Tea Leaf,The Coffee Bean and Tea Leaf,15.164039,120.609505,"Aniceto Gueco St, Angeles"
2,ChIJr_LhvYDtljMRoHrJ5BjoZJE,Coffee Bean and Tea Leaf,The Coffee Bean & Tea Leaf,15.168922,120.580243,"G/F SM City Clark, Manuel A. Roxas Hwy, Clark ..."
3,ChIJc0t13u3tljMRLbLEQRp3vZs,Coffee Bean and Tea Leaf,The Coffee Bean & Tea Leaf,15.169827,120.578192,"Tech Hub, SM Clark, Clark Freeport, Angeles"
4,ChIJ2dePHIjzljMRN2gsqVkO6S4,Coffee Bean and Tea Leaf,The Coffee Bean and Tea Leaf,15.167694,120.564239,"Clark Freeport, Mabalacat"
5,ChIJy19IElXtljMRnnAcNA-x2iA,Starbucks,Starbucks,15.177889,120.530057,"Manuel A. Roxas Hwy, Clark Freeport, Mabalacat"


This is important point: Bokeh does not understand Geometry data type. Recall that in the past articles, we created the `Geometry` variable from separate longitude and latitude data points. In this case, we need not do this step. For Bokeh, longitude and latitude should be separated.

To use bokeh for this purpose, we need to convert the dataframe into a format that Bokeh can understand ~ `ColumnDataSource`.

In [ ]:
# Point DataSource
# bokeh_source = ColumnDataSource(df)
# bokeh_source

Since we want to replicate the output we had with the other articles, let us use CartoDB Positron for this case. Because the longitude and latitude uses a different projection scale, we need to convert these before inputting as parameter. 

The set of coordinates below ensures that our map bounds are set to center the Philippines. 

In [5]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

ph_lon1, ph_lat1 = transform(outProj,inProj,115,0)
ph_lon2, ph_lat2 = transform(outProj,inProj,130,25)

In [9]:
# Initialize the Figure

cartodb = get_provider(CARTODBPOSITRON)

fig = figure(plot_width=800, plot_height=700,
             x_range=(ph_lon1, ph_lon2),
             y_range=(ph_lat1, ph_lat2),
             x_axis_type="mercator", 
             y_axis_type="mercator",
             tooltips=[
                    ("Cofee Brand", "@brand"), ("Location", "@vicinity")
                    ],
            title="Coffee Shops in the Philippines")

fig.add_tile(cartodb)

fig.xaxis.visible = False 
fig.yaxis.visible = False

show(fig)

We need to also convert the longitude and latitude per coffee shop as well.

In [7]:
lons, lats = [], []
for lon, lat in list(zip(df["lng"], df["lat"])):
    x, y = transform(outProj,inProj,lon,lat)
    lons.append(x)
    lats.append(y)
    
df["MercatorX"] = lons
df["MercatorY"] = lats

In [10]:
# Add the points to the map from our 'psource' ColumnDataSource -object
#Note that that the Latitude is the Y-axis, and the Longitude is the X-axis
fig.circle('MercatorX', 'MercatorY', 
           source=df, 
           size=7,
           fill_color='red',
           line_color='red',
           line_alpha=0.5,
             fill_alpha=0.3)
show(fig)

Now, as what we have done with most packages, we need to treat the different brands as a different dataset so the colors would be different per brand.  

In [12]:
color_dict = {
    "Starbucks": ' #00704A',
    "Coffee Bean and Tea Leaf": '#362d26',
    "Coffee Project": '#654321',
    "Tim Hortons": '#dd0f2d'
}

In [13]:
# from bokeh.io import curdoc

# curdoc().clear()

#Doing the Fig
fig = figure(plot_width=800, plot_height=700,
             x_range=(ph_lon1, ph_lon2),
             y_range=(ph_lat1, ph_lat2),
             x_axis_type="mercator", 
             y_axis_type="mercator",
             tooltips=[
                    ("Cofee Brand", "@brand"), ("Location", "@vicinity")
                    ],
            title="Coffee Shops in the Philippines")

fig.add_tile(cartodb)

fig.xaxis.visible = False 
fig.yaxis.visible = False

#Looping over the dataset
for i in color_dict.keys():
    temp = df[df.brand==i]
    fig.circle('MercatorX', 'MercatorY', 
           source=temp, 
           size=7,
           fill_color=color_dict[i],
           line_color=color_dict[i],
           line_alpha=0.5,
             fill_alpha=0.5)

In [15]:
pn.pane.Bokeh(fig)

Bokeh(Figure)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "fill_color" value " #00704A", key "line_color" value " #00704A" [renderer: GlyphRenderer(id='1566', ...)]
